<DIV ALIGN=CENTER>

# Introduction to Social Media: Twitter
## Professor Robert J. Brunner
  
</DIV>  
-----
-----


## Introduction


When looking for data to use for text data processing, one of the more
popular data sources is [Twitter][tw]. In this Notebook, we introduce
the Twitter API, and demonstrate how to use the Twitter API from within
a Python program to acquire and process tweets, or Twitter messages.


In this IPython Notebook, we explore 

OAUTH

Twitter client 

Process twitter information. SHow Tweet JSON.

Discuss rate limits

Sentiment analysis?

-----
[tw]: https://www.twitter.com

## Python and Twitter

To work with the Twitter API from within a Python program, we need a
Python library that wraps the official [Twitter API][twapi]. There are a
number of different Python libraries that provide this capability, we
will use the [tweepy][tpy] library, which is fairly popular and provides
a fairly complete interface.

The full Twitter API is large and robust (and continuous to evolve),
for this course we will restrict our attention to several basic
concepts, namely authenticating to Twitter, searching for Tweets, and
digesting the messages.

----
[twapi]: https://dev.twitter.com
[tpy]: http://www.tweepy.org

In [ ]:
import tweepy as tw

-----

## Reading Twitter Data

To read twitter data, you need to first need to be a registered Twitter
user and you need to create a new _Twitter Application_ in order to
obtain credentials for connecting to Twitter and querying to the
Twitter data. You create (and later manage) Twitter applications by
visting the [Twitter Application Management](https://apps.twitter.com)
website.

![Twitter App Sign-in](images/twitter-app-signin.png)

At this point you need to authenticate with Twitter, if you are already
logged in to Twitter on your computer (for instance by using the Twitter
website) you should already be authenticated. If you are not
authenticated, click the _sign in_ link to be directed to the Twitter
signin page where you can enter your credentials (if you do not have
Twitter credentials, you will need to obtain a Twitter account to
proceed).

![Twitter Sign-in](images/twitter-signin.png)

After you have been authenticated, you will be redirected to the Twitter
apps page. If you have never created a Twitter application, you will
have nothing listed. To create a new application, press the _Create New
App_ button, as shown in the following screenshot.

![Twitter Create App](images/twitter-create.png)

This will open up the Twitter _Create an application_ webpage, where you
need to supply some basic information for your Twitter application such
as an application name, description, and website.

![Twitter Application details](images/twitter-appdetails.png)

Scroll to the bottom of this webpage where the **Developer Agreement**
is located. Following this agreement, is a check box that you should
click to signify you agree to be bound by the agreement (of course you
should read this to be sure you do _agree_ with it first). Following
this, press the _Create your Twitter application_ button as shown in the
following screenshot.

![Twitter Agree](images/twitter-agree.png)

This will create your new application, and provide you with your
application webpage, which will be similar to the following screenshot.

![Twitter Apppage](images/twitter-apppage.png)

While you can control a number of application features from this
webpage, the most important tasks to complete include:

1. Change your application to _read-only_ in case it is set to
read-write.

2. Obtain the application **Consumer Key** and **Consumer Secret**.

3. Obtain your personal **Access Token** and **Access Token Secret**.

You should change your application read-only to ensure you don't
accidentally send data out to Twitter. You change this by selecting the
_Permissions_ tab and selecting _Read only_, shown in the following
screenshot. To save this setting, scroll down this webpage and click the
_Update Settings_ button at the bottom of the page.

![Twitter Read Only Setting](images/twitter-ro.png)

These credentials can be found by selecting the _Keys and Access Tokens_
tab, and scrolling down appropriately as shown in the following two
screenshots.

![Twitter Consumer Application Credentials](images/twitter-consume.png)

![Twitter User Credentials](images/twitter-access.png)

<font color='red'>Warning: Never share these credentials with others or
they will be able to fully impersonate you on Twitter!</font>

You can directly copy these credentials into your Notebook, or,
alternatively, save them into a file (for example by opening a terminal
window and using `vim` to create a text file. In the rest of this
Notebook, I demonstrate this functionality by using my credentials,
which I have saved into a file called`twitter.cred'. In this empty file,
which is in your github repository, I have saved the following four
credentials in order:

1. Access Token
2. Access Token Secret
3. Consumer Key
4. Consumer Secret

The following code cell demonstrates how these credentials are read from
the file and used to properly authenticate our application with Twitter.

-----




In [ ]:
tokens = []

# Order: Access Token, Access Token Secret, Consumer Key, Consumer SecretAccess

with open("twitter.cred", 'r') as fin:
    for line in fin:
        if line[0] != '#': # Not a comment line
            tokens.append(line.rstrip('\n'))

auth = tw.OAuthHandler(tokens[2], tokens[3])
auth.set_access_token(tokens[0], tokens[1])

api = tw.API(auth)

user = api.me()

print("Twitter Screen Name: ", user.screen_name)
print("Twitter Follower Count: ", user.followers_count)

print("\nThis user follows:\n--------------")
for friend in user.friends():
    print(friend.screen_name)

-----

If the following code cell runs without an error, you have successfully
connected to twitter. If you are new to twitter and are not following
anyone, you can instead display the user information for a different
Twitter user. For example, the following code would display my Twitter
information.

```python
user = api.get_user('ProfBrunner')
```

Replacing `ProfBrunner` with any valid Twitter user id will display
their information. You can find examples by looking at those Twitter
users you (or `ProfBrunner`) follow.

At any point, you can return to your Twitter application management
webpage to view your new application. You can now view and manage your
existing application, or create a new application as shown in the
following screenshot.

![Twitter new app management](images/twitter-manage.png)


-----


-----

## Student Activity

In the preceding cells, we used . Now that you
have run the Notebook, go back and make the following changes to see how
the results change.

1. Change the ?
2. Change the ?
3. Try making a ?

Finally, try applying a (discuss in class forums?

you should work through the previous Twitter
application setup in order to better learn how Twitter and in particular
the Tweepy Python library works. In addition, this will guarantee you
can follow along with the rest of this Notebook. Specific problems you
can attempt include the following:

1. Create a New Twitter application.

2. Save your Twitter credentials and Application credentials into the
provided `twitter.cred` file.

3. Run the _tweepy_ sample code to connect to Twitter and display your
Twitter user information.

Additional, more advanced problems:

1. Run the Twitter example above but for `ProfBrunner` instead.

2. Find the Twitter username for someone else (perhaps someone famous,
or someone else you know) and run the example code using their name.



-----

### Obtaining Tweets

Once you have authenticated with Twitter, you can begin to [search the
Twitter stream][stw] for tweets of interest. The easiest method to get started
is to being with your own (or another specific Twitter user's) own
Twitter feed. To access your own Twitter feed, you can simply use your
`home_timeline` to retrieve your own Tweets or Tweets from those whom
you follow. This is demonstrated in the following code cell, where we
display the `text` values from the ten most recent Tweets from our
timeline.

-----
[stw]: https://dev.twitter.com/rest/public/search

In [ ]:
for status in tw.Cursor(api.home_timeline).items(10):
    # Process a single status
    print(status.text) 

-----

### Searching

Twitter also provides the capability to search for specific tweets by
using the Tweepy [`search` method][twse]. In this method, you supply a
query string (and optional arguments) and are returned a list of Tweets.
The query string should follow the [Twitter Search API][tsa], but
basically you can search for specific text in a string by using the text
of interest, you can search for a person by using the `@` character
followed by their Twitter username, and hashtags by using the `#`
character followed by the tag text.

-----

[twse]: http://docs.tweepy.org/en/stable/api.html#API.search
[tsa]: https://dev.twitter.com/rest/public/search

In [ ]:
term ='UIResearchPark'

messages = []

current_page = 1
max_pages = 2

while(current_page <= max_pages):
    tweets = api.search(term, rpp=5)
    for tweet in tweets:
        messages.append(tweet)
    current_page += 1

for message in messages:
    print("Tweet ID:", message.id)
    print('Tweeted by ', message.user.screen_name)
    print("Created at ",message.created_at)
    print("Location: ",message.source)
    print('Tweet Text: ', message.text)
    print('-------------------------')



-----

We can view the available attributes to display by using Python `dir`
method to perform introspection. In the following code cell we
explicitly remove _class_ methods to minimize the display list and focus
on the items of interest. After this, we display the Tweet in its raw
JSON format by accessing the `_json` attribute.

-----

In [ ]:
[att for att in dir(message) if '__' not in att]

In [ ]:
# We can display the message data in JSON format
message._json